In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time

from tqdm import tqdm

In [2]:
import tibetcm_utils
import pandas as pd

## Here we will extract all the articles links from  https://www.tibetcm.com/ webpage
URL:  https://www.tibetcm.com/

Tabs not extracted

 https://www.tibetcm.com/


------------
-----------
------------
------------
------------

In [3]:
def loop_article_page(total_page, custom_url, key_code, base_url):
    """
    
    """
    return_file = {
        "Data": [],
        "message": "success",
        "response": 200
    }
    All_url_links = {}
    
    try:
        for i in tqdm(range(1, total_page+1)):
            if i == 1:
                final_url = custom_url + ".html"
            else:
                final_url = custom_url+ "_" + str(i) + ".html"
                
            found_url_links = tibetcm_utils.extract_all_tibetcm_page_article_links(final_url, base_url)
            key = key_code + str(i)
            All_url_links[key] = found_url_links
        return_file["Data"] = All_url_links
        return return_file
    
    except Exception as e:
        return_file["Data"] = All_url_links
        return_file["message"] = e
        return_file["response"] = 404
        return return_file

In [4]:
def check_error_in_links(All_url_link, page_code, print_each_error=False):
    """
    
    
    """

    error_counter = 0
    for page_id in range(1, len(All_url_link)):
        page_key = page_code + str(page_id)
        try:
            All_url_link.get(page_key)
            if  All_url_link.get(page_key)["Response"]!= 200:
                error_counter += 1
                if print_each_error:
                    print(page_key, All_url_link.get(page_key)["message"])
        except Exception as e:
            print(page_key, e)

    print(f"Total error in {page_code}: {error_counter}")

In [5]:
def save_json(path, file_name, data):
    """
    
    """
    with open(path+file_name, "w") as outfile:
        json.dump(data, outfile, indent=4, ensure_ascii=False)
        print(f"Successfully saved: {file_name}")

### all data

In [6]:
df = pd.read_csv("./data/All_cate.csv")
df.head()

,name_of_cate,original_url,Last_page_link,list_page_no
0,མདུན་ཤོག,https://www.tibetcm.com/,no link last page link,1
1,འཕྲིན་གསར།,https://www.tibetcm.com/news/,https://www.tibetcm.com/news/index_27.html,27
2,ལེགས་རྩོམ།,https://www.tibetcm.com/classics/,no link last page link,1
3,གཞུང་།,https://www.tibetcm.com/classics/gzhung/,no link last page link,1
4,བསྟོད་ཚོགས།,https://www.tibetcm.com/classics/praise/,https://www.tibetcm.com/classics/praise/index_...,3


In [9]:
sample_df = df.iloc[1:-1]
sample_df

,name_of_cate,original_url,Last_page_link,list_page_no
1,འཕྲིན་གསར།,https://www.tibetcm.com/news/,https://www.tibetcm.com/news/index_27.html,27
2,ལེགས་རྩོམ།,https://www.tibetcm.com/classics/,no link last page link,1
3,གཞུང་།,https://www.tibetcm.com/classics/gzhung/,no link last page link,1
4,བསྟོད་ཚོགས།,https://www.tibetcm.com/classics/praise/,https://www.tibetcm.com/classics/praise/index_...,3
5,མགུར་གླུ།,https://www.tibetcm.com/classics/poem-songs/,https://www.tibetcm.com/classics/poem-songs/in...,3
...,...,...,...,...
68,གཞན་དག,https://www.tibetcm.com/other/,no link last page link,1
69,ཡིག་ཚགས།,https://www.tibetcm.com/other/archives/,no link last page link,1
70,མི་རབས་གསུམ་པ།,https://www.tibetcm.com/other/third-poets/,/other/third-poets/index_3.html,3
71,མི་རབས་བཞི་བ།,https://www.tibetcm.com/other/fourth-poets/,no link last page link,1


In [10]:
for index, row in sample_df.iterrows():
    # print(row)
    name_of_cate = row["name_of_cate"]
    original_url = row["original_url"]
    Last_page_link = row["Last_page_link"]
    list_page_no = row["list_page_no"]

    total_page = list_page_no
    custom_url= original_url + "index"
    base_url = "https://www.tibetcm.com"
    article_tag = name_of_cate
    key_code = "Page " + article_tag + " "
    all_links = loop_article_page(total_page, custom_url, key_code, base_url)
    
    print(f"Total page in {article_tag}: {len(all_links['Data'])}")
    check_error_in_links(all_links['Data'], key_code, print_each_error=True)

    # Saving the final file
    path = "./data/links/"
    file_name = f"tibetcm_ALL_link_{article_tag}.json"
    save_json(path, file_name, all_links['Data'])
    

100%|██████████| 27/27 [00:29<00:00,  1.08s/it]


Total page in འཕྲིན་གསར།: 27
Total error in Page འཕྲིན་གསར། : 0
Successfully saved: tibetcm_ALL_link_འཕྲིན་གསར།.json


100%|██████████| 1/1 [00:02<00:00,  2.15s/it]


Total page in ལེགས་རྩོམ།: 1
Total error in Page ལེགས་རྩོམ། : 0
Successfully saved: tibetcm_ALL_link_ལེགས་རྩོམ།.json


100%|██████████| 1/1 [00:00<00:00,  1.09it/s]


...

Total page in མི་རབས་གསུམ་པ།: 3
Total error in Page མི་རབས་གསུམ་པ། : 0
Successfully saved: tibetcm_ALL_link_མི་རབས་གསུམ་པ།.json


100%|██████████| 1/1 [00:00<00:00,  1.11it/s]


Total page in མི་རབས་བཞི་བ།: 1
Total error in Page མི་རབས་བཞི་བ། : 0
Successfully saved: tibetcm_ALL_link_མི་རབས་བཞི་བ།.json


100%|██████████| 1/1 [00:00<00:00,  1.10it/s]

Total page in ཚོང་བརྡ།: 1
Total error in Page ཚོང་བརྡ། : 0
Successfully saved: tibetcm_ALL_link_ཚོང་བརྡ།.json
